In [35]:
import pandas as pd 
import requests
from datetime import datetime
import time

## collect_coingecko_prices

In [5]:
session = requests.Session()

In [6]:
coin_ids=["bitcoin", "ethereum", "cardano", "solana"]

In [7]:
ids_str = ",".join(coin_ids)

In [9]:
url = "https://api.coingecko.com/api/v3/simple/price"
params = {
            "ids": ids_str,
            "vs_currencies": "usd,eur,btc",
            "include_market_cap": "true",
            "include_24hr_vol": "true",
            "include_24hr_change": "true",
            "include_last_updated_at": "true",
        }

In [10]:
response = session.get(url, params=params, timeout=10)

In [11]:
response.status_code

200

In [12]:
data = response.json()

In [19]:
timestamp = datetime.utcnow()

In [20]:
enriched_data = {
            "collected_at": timestamp.isoformat(),
            "source": "coingecko",
            "data_type": "spot_prices",
            "prices": data,
        }

## collect_coingecko_market_data

In [21]:
vs_currency = "usd"
per_page = 100

url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
            "vs_currency": vs_currency,
            "order": "market_cap_desc",
            "per_page": per_page,
            "page": 1,
            "sparkline": "false",
            "price_change_percentage": "1h,24h,7d",
        }

In [22]:
response = session.get(url, params=params, timeout=15)

In [23]:
response.status_code

200

In [24]:
data = response.json()

In [26]:
len(data)

100

In [27]:
df = pd.DataFrame(data)

## collect_binance_ticker

In [29]:
symbols=["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT"]

In [34]:
url = "https://api.binance.com/api/v3/ticker/24hr"


In [36]:
all_tickers = []
for symbol in symbols:
    url = "https://api.binance.com/api/v3/ticker/24hr"
    response = session.get(url, params={"symbol": symbol}, timeout=10)
    response.raise_for_status()
    all_tickers.append(response.json())
    time.sleep(0.1)

In [37]:
timestamp = datetime.utcnow()
enriched_data = {
    "collected_at": timestamp.isoformat(),
    "source": "binance",
    "data_type": "24hr_ticker",
    "tickers": all_tickers,
}


## collect_binance_klines

In [39]:
url = "https://api.binance.com/api/v3/klines"

In [41]:
symbol="BTCUSDT"
interval="1h"
limit=100

In [42]:
params = {"symbol": symbol, "interval": interval, "limit": limit}

In [43]:
response = session.get(url, params=params, timeout=15)

In [45]:
data = response.json()

In [48]:
df = pd.DataFrame(data, columns=[
            "open_time", "open", "high", "low", "close", "volume",
            "close_time", "quote_volume", "trades", "taker_buy_base",
            "taker_buy_quote", "ignore"
        ])

## collect_fear_greed_index

In [52]:
url = "https://api.alternative.me/fng/"
response = session.get(url, params={"limit": 10}, timeout=10)
response.raise_for_status()
data = response.json()

In [53]:
data

{'name': 'Fear and Greed Index',
 'data': [{'value': '60',
   'value_classification': 'Greed',
   'timestamp': '1759881600',
   'time_until_update': '81282'},
  {'value': '70', 'value_classification': 'Greed', 'timestamp': '1759795200'},
  {'value': '71', 'value_classification': 'Greed', 'timestamp': '1759708800'},
  {'value': '74', 'value_classification': 'Greed', 'timestamp': '1759622400'},
  {'value': '71', 'value_classification': 'Greed', 'timestamp': '1759536000'},
  {'value': '63', 'value_classification': 'Greed', 'timestamp': '1759449600'},
  {'value': '64', 'value_classification': 'Greed', 'timestamp': '1759363200'},
  {'value': '49',
   'value_classification': 'Neutral',
   'timestamp': '1759276800'},
  {'value': '50',
   'value_classification': 'Neutral',
   'timestamp': '1759190400'},
  {'value': '50',
   'value_classification': 'Neutral',
   'timestamp': '1759104000'}],
 'metadata': {'error': None}}

## collect_coinbase_orderbook

In [54]:
product_id="BTC-USD"
level=2

url = f"https://api.exchange.coinbase.com/products/{product_id}/book"

In [55]:
response = session.get(url, params={"level": level}, timeout=10)

In [56]:
data = response.json()